In [17]:
import sys
# %store -r SYSPATHS
SYSPATHS = {"tool":"../../../tool"}
sys.path.append(SYSPATHS['tool'])
import Khfeature as kf
import Khcore1 as kc1
import MyEcharts as me
import pandas as pd
import numpy as np
import pylab as pl
import os
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn import linear_model
import pickle

In [2]:
step = "1min"  ## 计算步进间隔
roll_window = '1H'  ## 滑动计算窗口
slope_window = "4H"  ## 斜率计算窗口

## 一、浓缩段温度异常工况故障树梳理

<table>
	<tr>
    <th>异常工况</th>
    <th>故障现象</th>
    <th>故障一级原因</th>
    <th>故障二级原因</th>
    <th>备注</th>
   </tr>

  <tr>
    <td rowspan="7" valign="middle" align="middle">浓缩段温度超标</td>
    <td rowspan="7" valign="middle" align="middle">浓缩段温度升高</td>
   </tr>
   
  <tr>
    <td valign="middle" align="middle">入口负荷增加</td>
    <td valign="middle" align="middle">原烟气温度升高</td>
    <td valign="middle" align="middle">对应测点 原烟气温度</td>
   <tr>
       
   <tr>
    <td valign="middle" align="middle">处于两个浓缩泵切换过程</td>
    <td> </td>
    <td valign="middle" align="middle">对应测点 浓缩泵A电流<br>浓缩泵B电流</td>
   </tr>
   
   <tr>
    <td valign="middle" align="middle">脱硫塔固含量升高</td>
    <td valign="middle" align="middle">处于出料期间</td>
    <td valign="middle" align="middle">对应测点 蒸汽母管流量<br>
塔底浆液液位<br>
固含量可以通过检测化验方式获取</td>
   </tr>
   
   <tr>
    <td valign="middle" align="middle">入口烟气含高温水</td>
    <td valign="middle" align="middle">锅炉爆管导致</td>
    <td valign="middle" align="middle">对应锅炉段测点</td>
   </tr>
</table>

## 二、获取自变量数据

In [4]:
r = []
node = ['TE_1003a','TE_1003b','TE_1003c',"PT_4001",'P2002A_AI','P2002B_AI','PT_2003','P2003A_AI','TE_1002','TE_1004']
cols = ['浓缩段温度A','浓缩段温度B','浓缩段温度C','蒸汽压力','浓缩泵A电流',\
        '浓缩泵B电流','浓缩泵出口压力','排出泵A电流','塔原烟气温度','塔吸收段温度']
for n in node:
    r1 = kc1.ReadRawData(n)
    r1= r1[r1.index>"2019-06-13 19:00:00"] 
    r.append(r1)
r= pd.concat(r,axis=1)
r = r.resample(step).mean()
r.columns = cols
temp = pd.read_pickle("../../workspace/Dzy/data/process/ZC_06.pk")
r = pd.concat([r,temp[['label', 'Logistic']]],axis=1) ## 加入自动标签和 预测结果
r['浓缩泵电流'] = (r['浓缩泵A电流'].pow(2)+r['浓缩泵B电流'].pow(2)).pow(0.5)
r.head()

,浓缩段温度A,浓缩段温度B,浓缩段温度C,蒸汽压力,浓缩泵A电流,浓缩泵B电流,浓缩泵出口压力,排出泵A电流,塔原烟气温度,塔吸收段温度,label,Logistic,浓缩泵电流
time,,,,,,,,,,,,,
2019-06-13 19:00:00,61.355300,59.926700,59.816900,0.0,0.293000,190.378500,0.452300,0.415100,150.525000,55.274700,0.0,0.579946,190.378725
2019-06-13 19:01:00,61.558003,59.875460,59.851057,0.0,0.293000,190.144067,0.452220,0.408593,150.518510,55.275943,0.0,0.572139,190.144292
2019-06-13 19:02:00,61.684993,59.822970,59.844953,0.0,0.247453,190.215693,0.451653,0.416733,150.476213,55.212473,0.0,0.598396,190.215854
2019-06-13 19:03:00,61.789990,59.888887,59.896207,0.0,0.195400,190.306863,0.451903,0.415100,150.466440,55.227123,0.0,0.625033,190.306964
2019-06-13 19:04:00,61.796087,59.901080,59.914513,0.0,0.283240,190.202680,0.452017,0.415100,150.446917,55.213703,0.0,0.651563,190.202891


In [5]:
r1 = (r-r.min())/(r.max()-r.min())
me.Plot_LineBar(r1,name="reason",overwrite=True)

#### 数据观察结论：
    1. 排出泵电流 和 蒸汽压力 都意味着处于出料期间, 但是 排出泵电流的变化 不如 蒸汽压力变化敏感，所以应以蒸汽压力作为出料期间的判定变量
    2. 原烟气入口温度变化 波动较大
    3. 处于出料期间时的 从 浓缩泵电流变化趋势也能体现，且浓缩泵电流趋势变化早于 蒸汽压力
    4. 浓缩段出口压力 和 浓缩段循环泵电流变化趋势一致, 且其变化较电流 更加平稳, 但是压力没有A,B之分, 可以作为区分出料的特征, 但是无法代替电流识别 泵切换问题

## 三、加工原因变量的特征

In [6]:
### 1. 浓缩泵切换 : 已在吉林大成 和 宁夏尹品测试该特征加工的有效性
r1 = r[['蒸汽压力']]
r1['浓缩泵切换'] = (r['浓缩泵电流']-(r['浓缩泵B电流']-r['浓缩泵A电流']).abs()).abs()
r1['浓缩泵切换'] = (r1['浓缩泵切换']>1.2)*1
r1['蒸汽压力'] = (r1['蒸汽压力']>0.1)*1
s0 = kf.SingleTsFeature(r["浓缩泵出口压力"])
s0.add_roll_quantile(q=0.75,varname="泵压力值",window=roll_window)
s0.add_slope(varname="泵斜率",window=slope_window)
r1 = pd.concat([r1,s0.GetDf(),r[['Logistic']]],axis=1)

/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

add 泵压力值 successful 
add_slope successful 


In [7]:
temp = (r1-r1.min())/(r1.max()-r1.min())
me.Plot_LineBar(temp,name="temp",overwrite=True)

## 二、标记原因标签
0. 正常
1. 处于出料期间
2. 浓缩泵切换
3. 浓缩泵异常
4. 原烟气温度异常
5. 其他

In [8]:
r2 = r1[['蒸汽压力', '浓缩泵切换', '泵压力值', '泵斜率']]

### 观察浓缩泵出口压力 处于下降且不属于出料期间的 标记为 浓缩泵异常
bins = [['2019-06-14 06:36','2019-06-14 10:28'],['2019-06-23 08:21','2019-06-23 13:28'],\
        ['2019-06-25 22:40','2019-06-26 05:46'],['2019-06-27 07:19','2019-06-27 08:33'],\
        ['2019-06-28 10:04','2019-06-28 15:00'],['2019-06-30 00:28','2019-06-30 01:48']]
labels =['浓缩泵异常']*len(bins)
r2 = kc1.generate_label(r2,bins=bins,labels=labels)

### 观察蒸汽压力 为 1 的标记为 出料期间
r2.loc[r2[r2['蒸汽压力']==1].index,'label'] = '处于出料期间'

### 将Logistic 超过报警阈值, 且无标记的标记为 其他
temp = r2.loc[r[r['Logistic']>0.6].index,"label"]
r2.loc[temp[temp.isnull()].index,"label"] = "其他"

### 剩余标签 标记为 正常
r2['label']=  r2['label'].fillna('正常')

In [9]:
##  观看 原因标记效果
temp = pd.pivot_table(r2,index = 'time',columns='label',values="泵压力值",aggfunc = "first")
temp = pd.concat([temp,r2[['蒸汽压力', '浓缩泵切换']],r1[['Logistic']]],axis=1)
me.Plot_LineBar(temp,name="reason_auto_label",overwrite=True)

## 三、模型训练

### 1. 提取模型X,Y 

In [25]:
scaler = MinMaxScaler()
X= r2[['蒸汽压力', '浓缩泵切换', '泵压力值', '泵斜率']].values
X1=scaler.fit_transform(X)
Y = r2[['label']].values

In [26]:
## 保存原因模型的 scale
pickle.dump(scaler, open('./model/reason_scale.pkl', 'wb'))

### 2. 模型训练

In [27]:
from sklearn.neighbors.nearest_centroid import NearestCentroid
import numpy as np
clf = NearestCentroid()
clf.fit(X1, Y)
r2['predict'] = clf.predict(X1)

/root/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [28]:
## 保存原因模型的 model
pickle.dump(clf, open('./model/reason_model.pkl', 'wb'))

In [29]:
## 全域下的 模型预测精度
print ("全域下的 模型预测精度 : %0.2f%%"%(clf.score(X1,Y)*100))
temp = r2[r2.label!=r2.predict]
for label in temp.label.unique():
    print ('被错误分类的标签为 : %s  \n 模型预测为: '%label)
    print (temp[temp.label==label].predict.value_counts())
    print ("*"*30)

全域下的 模型预测精度 : 71.78%
被错误分类的标签为 : 正常  
 模型预测为: 
其他       5759
浓缩泵异常     548
Name: predict, dtype: int64
******************************
被错误分类的标签为 : 其他  
 模型预测为: 
正常       179
浓缩泵异常    155
Name: predict, dtype: int64
******************************
被错误分类的标签为 : 浓缩泵异常  
 模型预测为: 
其他    315
正常     37
Name: predict, dtype: int64
******************************


In [30]:
## 超过报警阈值下的 模型预测精度
temp = r2[(r1.Logistic>0.6)]
score = len(temp[temp.label==temp.predict])/len(temp)*100
print ("超过报警阈值下的 模型预测精度 : %0.2f%%"%(score))

超过报警阈值下的 模型预测精度 : 91.76%


In [16]:
##  观看 原因标记效果
temp = pd.pivot_table(r2,index = 'time',columns='label',values="泵压力值",aggfunc = "first")
temp = pd.concat([temp,r2[['蒸汽压力', '浓缩泵切换']],r1[['Logistic']]],axis=1)
temp['预测正确'] = (r2.label==r2.predict)*1
me.Plot_LineBar(temp,name="reason_auto_predict",overwrite=True)